In [17]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unidecode
from src.data import preprocessing
import os

In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
params = preprocessing.dict_params
INPUT_FILE_PATH = './../data/raw'
OUTPUT_FILE_PATH = './../data/processed'
INDEX_KEY = [0] # the 1st column is the primary key

### Clean column names (delete spaces, special characters, ...)

In [38]:
params = preprocessing.params
dfs = preprocessing._transform_column_names(INPUT_FILE_PATH, OUTPUT_FILE_PATH)

2021-11-21 15:42:51,757 - preprocessing - preprocess_dataset - INFO : Preprocessing dataset ./../data/raw/Agribalyse_Synthese.csv
2021-11-21 15:42:51,781 - preprocessing - preprocess_dataset - INFO : Preprocessing dataset ./../data/raw/Agribalyse_Detail ingredient.csv
2021-11-21 15:42:51,822 - preprocessing - preprocess_dataset - INFO : Preprocessing dataset ./../data/raw/Agribalyse_Detail etape.csv


In [39]:
dfs['ingredients'].head()

,Ciqual_AGB,Nom_Francais,Groupe_aliment,Sous-groupe_aliment,LCI_Name,Ingredients,Score_unique_EF_
0,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jaune d'œuf,0.039743
1,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Ail,0.004417
2,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jus de citron,0.019375
3,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Huile d'olive,0.445364
4,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Autres étapes,0.080761


### Generate a dataframe with statistics of ingredients

In [40]:
orig_ing_df = dfs['ingredients']
desc_ingred_df = preprocessing._get_describe_ingredients(orig_ing_df)
print(desc_ingred_df.shape)
desc_ingred_df.head(5)

(213, 3)


,Ingredients,min_EF,max_EF
0,Abats de bœuf,0.365335,1.976721
1,Abats de porc,0.008707,0.228660
2,Abats de poulet,0.003403,0.049212
3,Abats de veau,0.398916,0.398916
4,Abricot,0.004451,0.046987


### Merge original ingredients dataset with statistics (min, max)

In [41]:
ing_df = orig_ing_df.merge(desc_ingred_df, how='left', left_on='Ingredients', right_on='Ingredients') 
print(ing_df.shape)
ing_df.head(5)

(5671, 9)


,Ciqual_AGB,Nom_Francais,Groupe_aliment,Sous-groupe_aliment,LCI_Name,Ingredients,Score_unique_EF_,min_EF,max_EF
0,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jaune d'œuf,0.039743,0.006586,0.089649
1,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Ail,0.004417,0.000310,0.009735
2,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Jus de citron,0.019375,0.004003,0.088407
3,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Huile d'olive,0.445364,0.005854,0.764705
4,11168,"Sauce aïoli, préemballée",aides culinaires et ingrédients divers,sauces,"Aioli sauce (garlic and olive oil mayonnaise),...",Autres étapes,0.080761,0.005990,1.996034


### Create sparse matrix with ingredients

In [44]:
key_columns = ['Ciqual_AGB', 'Nom_Francais', 'Groupe_aliment', 'Sous-groupe_aliment', 'LCI_Name' , 'Ingredients']
pivot_column = 'Ingredients'
value_columns = 'Score_unique_EF_'
drop_cols = ['Nom_Francais', 'Groupe_aliment', 'Sous-groupe_aliment', 'LCI_Name']
sparse_ing_df = preprocessing._create_sparse_ingredients(ing_df,key_columns, pivot_column, value_columns)
sparse_ing_df.head(3)

,Ciqual_AGB,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,...,Viande de moutton sans os,Viande de porc maigre,Viande de poulet sans os,Viande de veau sans os,Vin blanc,Vin rouge,Yaourt,citron,Échalote,Œuf de poule
0,10023,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10042,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### Create sparse matrix where min_EF, max_EF of each ingredient are in columns

In [33]:
pivot_index = ['Ciqual_AGB', 'Nom_Francais', 'Groupe_aliment', 'Sous-groupe_aliment', 'LCI_Name']
pivot_columns = ['Ingredients']
pivot_values = ['min_EF', 'max_EF']
drop_cols = ['Nom_Francais', 'Groupe_aliment', 'Sous-groupe_aliment', 'LCI_Name']
pivot_ing_minmax = preprocessing._create_features_min_max_ing(ing_df, pivot_index, pivot_columns, pivot_values)
pivot_ing_minmax.head()

,Ciqual_AGB,min_EF_Abats de bœuf,min_EF_Abats de porc,min_EF_Abats de poulet,min_EF_Abats de veau,min_EF_Abricot,min_EF_Ail,min_EF_Amande,min_EF_Amidon de maïs,min_EF_Ananas,...,max_EF_Viande de moutton sans os,max_EF_Viande de porc maigre,max_EF_Viande de poulet sans os,max_EF_Viande de veau sans os,max_EF_Vin blanc,max_EF_Vin rouge,max_EF_Yaourt,max_EF_citron,max_EF_Échalote,max_EF_Œuf de poule
0,10023,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.676611
1,10042,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.009161,0.000000
2,1007,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.089332,0.0,0.0,0.0,0.000000,0.000000
3,10081,0.0,0.0,0.0,0.0,0.0,0.00031,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
4,10082,0.0,0.0,0.0,0.0,0.0,0.00031,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.089332,0.0,0.0,0.0,0.009161,0.000000


### Merge sparse ingredients data with pivoted min max values

In [45]:
drop_cols = ['Nom_Francais', 'Groupe_aliment', 'Sous-groupe_aliment', 'LCI_Name']
new_ing_df = sparse_ing_df.merge(pivot_ing_minmax, how = 'inner', left_on='Ciqual_AGB', right_on="Ciqual_AGB")
dfs['ingredients'] = new_ing_df
new_ing_df.head(5)

,Ciqual_AGB,Abats de bœuf,Abats de porc,Abats de poulet,Abats de veau,Abricot,Ail,Amande,Amidon de maïs,Ananas,...,max_EF_Viande de moutton sans os,max_EF_Viande de porc maigre,max_EF_Viande de poulet sans os,max_EF_Viande de veau sans os,max_EF_Vin blanc,max_EF_Vin rouge,max_EF_Yaourt,max_EF_citron,max_EF_Échalote,max_EF_Œuf de poule
0,10023,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.676611
1,10042,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.009161,0.000000
2,1007,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.089332,0.0,0.0,0.0,0.000000,0.000000
3,10081,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000
4,10082,0,0,0,0,0,1,0,0,0,...,0.0,0.0,0.0,0.0,0.089332,0.0,0.0,0.0,0.009161,0.000000


In [46]:
synthese_key = params.synthese.index_key.to_list()#get_param('synthese', 'index_key')

ingredients_key = params.ingredients.index_key.to_list() #get_param('ingredients', 'index_key')
etapes_key = params.etapes.index_key.to_list()#get_param('etapes', 'index_key')

new_ingred_df = preprocessing.merge_dataset(dfs['synthese'], dfs['ingredients'], synthese_key,ingredients_key)
new_etape_df = preprocessing.merge_dataset(dfs['synthese'], dfs['etapes'], synthese_key, etapes_key)
new_ingred_df.head(5)

,Code_AGB,Code_CIQUAL,Groupe_aliment,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Saisonnalite,Transport_par_avion_,Livraison,Materiau_emballage,...,max_EF_Viande de moutton sans os,max_EF_Viande de porc maigre,max_EF_Viande de poulet sans os,max_EF_Viande de veau sans os,max_EF_Vin blanc,max_EF_Vin rouge,max_EF_Yaourt,max_EF_citron,max_EF_Échalote,max_EF_Œuf de poule
0,13712,13712,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, égoutté","Apricot, canned in light syrup, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13713,13713,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, non égoutté","Apricot, canned in light syrup, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13714,13714,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, égoutté","Apricot, in syrup, canned, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13715,13715,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, non égoutté","Apricot, in syrup, canned, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13000,13000,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru","Apricot, pitted, raw",mix de consommation FR,0,Ambiant (moyenne),LPDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Clean nan values and save the cleaned files in OUPUT_FILE_PATH

In [47]:
new_ingred_df.fillna(0, inplace=True)
new_etape_df.fillna(0, inplace=True)

OUTPUT_FILE_PATH = './../data/processed'
new_ingred_df.to_csv(os.path.join(OUTPUT_FILE_PATH, params.ingredients.file_name), index=False)
new_etape_df.to_csv(os.path.join(OUTPUT_FILE_PATH, params.etapes.file_name), index=False)

In [13]:
new_ingred_df

,Code_AGB,Code_CIQUAL,Groupe_aliment,Sous-groupe_aliment,Nom_Produit_Francais,LCI_Name,Saisonnalite,Transport_par_avion_,Livraison,Materiau_emballage,...,max_EF_Viande de moutton sans os,max_EF_Viande de porc maigre,max_EF_Viande de poulet sans os,max_EF_Viande de veau sans os,max_EF_Vin blanc,max_EF_Vin rouge,max_EF_Yaourt,max_EF_citron,max_EF_Échalote,max_EF_Œuf de poule
0,13712,13712,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, égoutté","Apricot, canned in light syrup, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,13713,13713,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop léger, appertisé, non égoutté","Apricot, canned in light syrup, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13714,13714,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, égoutté","Apricot, in syrup, canned, drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13715,13715,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot au sirop, appertisé, non égoutté","Apricot, in syrup, canned, not drained",mix de consommation FR,0,Ambiant (moyenne),Acier,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,13000,13000,"fruits, légumes, légumineuses et oléagineux",fruits,"Abricot, dénoyauté, cru","Apricot, pitted, raw",mix de consommation FR,0,Ambiant (moyenne),LPDE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2474,19592,19592,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, wit...",mix de consommation FR,0,Glacé,PP,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2475,19593,19593,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, plain",mix de consommation FR,0,Glacé,PP,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2476,19594,19594,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, pla...",mix de consommation FR,0,Glacé,PP,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2477,19598,19598,lait et produits laitiers,produits laitiers frais et assimilés,"Yaourt, lait fermenté ou spécialité laitière, ...","Yogurt, fermented milk or dairy specialty, pla...",mix de consommation FR,0,Glacé,PP,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
